In [94]:
import pandas as pd
import numpy as np
import time

def get_delta(row):
    if (row['Method']=='send_file_to_filenode'):
        return 0
    t1 = row['Time']
    t2 = row['Id']
    t = t1-t2  # pandas._libs.tslibs.timedeltas.Timedelta
    t = t.seconds * 1000000 + t.microseconds
    return t/1000000

path = './data/mean_110ms_latency/50x10/'
#path = './data/mean_110ms_latency/25x1000/'
#path = './data/mean_110ms_latency/25x100000/'
path1 = path + 'nameNode.log'
path2 = path + 'client.log'

# Load name node data from csv file
data1 = pd.read_csv(path1, delimiter=';', header=None, skiprows=2)
data1.columns = ['LogTime', 'Time', 'Node', 'Method', 'Id'] 
data2 = pd.read_csv(path2, delimiter=';', header=None)
data2.columns = ['LogTime', 'Time', 'Node', 'Method', 'Id'] 
data = pd.concat([data1, data2])

# Manipulate
#data['Time'] = data['Time'].apply(lambda x: x.replace('_', ' '))
data['Time'] =  pd.to_datetime(data['Time'])
data['Id'] =  pd.to_datetime(data['Id'])
data['Method'] = data.apply(lambda row: ('0 step' if row['Node'] == 'Client' else ('1 step' if row['Method'] == 'update_filelist' else '2 step')), axis=1)
data = data.sort_values(['Id', 'Time'], ascending=[1, 1])
data['Delta'] = data.apply(get_delta, axis=1)

# Drop unnecessary columns
data = data.drop("Time", axis=1)
data = data.drop("LogTime", axis=1)

# Pivot
data = data.pivot(index='Id', columns='Method', values='Delta')
data['Total'] = data.sum(axis=1)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(data)
data

Method,0 step,1 step,2 step,Total
Id,,,,
2019-11-20 09:17:09.273766,0.0,0.753339,1.806210,2.559549
2019-11-20 09:17:11.617269,0.0,0.761208,1.795900,2.557108
2019-11-20 09:17:13.979049,0.0,0.758190,1.830392,2.588582
2019-11-20 09:17:16.313396,0.0,0.761038,1.861444,2.622482
2019-11-20 09:17:18.732180,0.0,0.748351,1.844486,2.592837
2019-11-20 09:17:21.088925,0.0,0.732435,1.771125,2.503560
2019-11-20 09:17:23.410690,0.0,0.733446,1.791296,2.524742
2019-11-20 09:17:25.734018,0.0,0.766063,1.792749,2.558812
2019-11-20 09:17:28.002048,0.0,0.717998,1.751762,2.469760


In [95]:
# First file list update
data['1 step'].describe()

count    50.000000
mean      0.750013
std       0.023479
min       0.696061
25%       0.732551
50%       0.750041
75%       0.769483
max       0.795315
Name: 1 step, dtype: float64

In [96]:
# Second file list update
data['2 step'].describe()

count    50.000000
mean      1.814501
std       0.039680
min       1.738792
25%       1.784294
50%       1.815526
75%       1.842472
max       1.923551
Name: 2 step, dtype: float64

In [97]:
# Total
data['Total'].describe()

count    50.000000
mean      2.564514
std       0.056237
min       2.434853
25%       2.523871
50%       2.560230
75%       2.603182
max       2.694170
Name: Total, dtype: float64

In [41]:
from datetime import datetime, timedelta
import random

started = datetime.now()
milliseconds = sorted(list(set([random.randint(10, 2000) for iter in range(110)])))[:100]
latencies1 = [random.randint(10, 2000) for iter in range(100)]
latencies2 = [random.randint(10, 2000) for iter in range(100)]

# Client log
for i in range(0,100):
    now = started + timedelta(milliseconds=milliseconds[i])
    print('%s;%s_%s;%s' % (now, 'Client', 'get_fileNode', now))

print('/n****************************************************/n')
    
# Name node log
for i in range(0,100):
    now = started + timedelta(milliseconds=milliseconds[i])
    # Name node 1
    print('%s;%s_%s;%s' % (now + timedelta(milliseconds=latencies1[i]), 'NameNode', 'post_fileList', now))
    # Name node 2
    print('%s;%s_%s;%s' % (now + timedelta(milliseconds=latencies1[i]+latencies2[i]), 'NameNode', 'post_fileList_replica', now))



2019-11-20 08:16:23.127994;Client_get_fileNode;2019-11-20 08:16:23.127994
2019-11-20 08:16:23.129994;Client_get_fileNode;2019-11-20 08:16:23.129994
2019-11-20 08:16:23.170994;Client_get_fileNode;2019-11-20 08:16:23.170994
2019-11-20 08:16:23.193994;Client_get_fileNode;2019-11-20 08:16:23.193994
2019-11-20 08:16:23.195994;Client_get_fileNode;2019-11-20 08:16:23.195994
2019-11-20 08:16:23.196994;Client_get_fileNode;2019-11-20 08:16:23.196994
2019-11-20 08:16:23.204994;Client_get_fileNode;2019-11-20 08:16:23.204994
2019-11-20 08:16:23.219994;Client_get_fileNode;2019-11-20 08:16:23.219994
2019-11-20 08:16:23.224994;Client_get_fileNode;2019-11-20 08:16:23.224994
2019-11-20 08:16:23.235994;Client_get_fileNode;2019-11-20 08:16:23.235994
2019-11-20 08:16:23.258994;Client_get_fileNode;2019-11-20 08:16:23.258994
2019-11-20 08:16:23.270994;Client_get_fileNode;2019-11-20 08:16:23.270994
2019-11-20 08:16:23.324994;Client_get_fileNode;2019-11-20 08:16:23.324994
2019-11-20 08:16:23.377994;Client_get_